In [1]:
import os
import matplotlib.pyplot as plt
# print(os.listdir(('/content/drive/MyDrive/GlauPro')))

import datetime as datetime
import time
import seaborn as sns

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
import tensorflow as tf
from keras.optimizers import SGD
from tensorflow import keras
import keras.layers as layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow.keras.backend as K
import numpy as np
import itertools
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


## Model Setup

In [3]:
input_shape=(256, 256, 3)

### Vanilla CNN

In [4]:
# Define the CNN vanilla_model without regularization
vanilla_model = Sequential()
vanilla_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
vanilla_model.add(Conv2D(64, (3, 3), activation='relu'))
vanilla_model.add(MaxPooling2D(pool_size=(2, 2)))
vanilla_model.add(Dropout(0.25))
vanilla_model.add(Flatten())
vanilla_model.add(Dense(128, activation='relu'))
vanilla_model.add(Dropout(0.5))
vanilla_model.add(Dense(1, activation='sigmoid'))

vanilla_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

vanilla_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 126, 126, 64)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 126, 126, 64)      0         
                                                                 
 flatten (Flatten)           (None, 1016064)           0         
                                                                 
 dense (Dense)               (None, 128)               130056320 
                                                        

### ALEXNet

In [5]:
# AlexNet model
class AlexNet(Sequential):
    def __init__(self, num):
        super().__init__()

        self.add(Conv2D(96, kernel_size=(11,11), strides= 4,
                        padding= 'valid', activation= 'relu',
                         input_shape=input_shape,
                        kernel_initializer= 'he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Conv2D(256, kernel_size=(5,5), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None)) 

        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Conv2D(256, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Flatten())
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(1000, activation= 'relu'))
        self.add(Dense(num, activation= 'sigmoid'))

        #self.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['categorical_accuracy'])
        
        # opt = SGD(lr=0.01, momentum=0.9)
        self.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])
        
    @classmethod
    def from_config(cls, config):
        sublayer_config = config.pop("sublayer")
        sublayer = keras.saving.deserialize_keras_object(sublayer_config)
        return cls(sublayer, **config)

cls = 1
alexnet_model = AlexNet(cls)
alexnet_model.summary()

Model: "alex_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 96)        34944     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 96)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 30, 30, 256)       614656    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 384)       885120    
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 384)       132

### GoogleNet

In [6]:
# Sequential Model Definition
def create_googlenet_model(num):
    model = Sequential()
    inputs = keras.Input(shape=input_shape, name='Input')
    x = layers.Conv2D(32, 3, padding='same', kernel_regularizer = regularizers.l2(0.01),
                    name='Conv_1')(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, padding='same', kernel_regularizer = regularizers.l2(0.01),
                    name='Conv_2')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)


    x = layers.Conv2D(128, 3, padding='same', kernel_regularizer = regularizers.l2(0.01), name='Conv_3')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(32, 3, padding='same', kernel_regularizer = regularizers.l2(0.01), name='Conv_4')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)

    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer = regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(num)(x)
    model = keras.Model(inputs=inputs, outputs=output)
    
    #opt = SGD(lr=0.01, momentum=0.9)
    model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])

    return model
cls = 1
googlenet_model = create_googlenet_model(cls)
googlenet_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 256, 256, 3)]     0         
                                                                 
 Conv_1 (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 256, 256, 32)      128       
 Normalization)                                                  
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 256, 256, 32)      0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 128, 128, 32)      0         
 g2D)                                                            
                                                                 
 Conv_2 (Conv2D)             (None, 128, 128, 64)      18496 

### ResNet

In [7]:
res_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

x = res_model.output
x = GlobalAveragePooling2D()(x)
output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification output

resnet_model = Model(inputs=res_model.input, outputs=output_layer)

#opt = SGD(lr=0.01, momentum=0.9)
resnet_model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])

resnet_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

                                                                                                  
 conv2_block3_1_conv (Conv2  (None, 64, 64, 64)           16448     ['conv2_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv2_block3_1_bn (BatchNo  (None, 64, 64, 64)           256       ['conv2_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv2_block3_1_relu (Activ  (None, 64, 64, 64)           0         ['conv2_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv2_blo

 ation)                                                                                           
                                                                                                  
 conv3_block2_3_conv (Conv2  (None, 32, 32, 512)          66048     ['conv3_block2_2_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block2_3_bn (BatchNo  (None, 32, 32, 512)          2048      ['conv3_block2_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block2_add (Add)      (None, 32, 32, 512)          0         ['conv3_block1_out[0][0]',    
                                                                     'conv3_block2_3_bn[0][0]']   
          

 conv4_block1_2_conv (Conv2  (None, 16, 16, 256)          590080    ['conv4_block1_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv4_block1_2_bn (BatchNo  (None, 16, 16, 256)          1024      ['conv4_block1_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block1_2_relu (Activ  (None, 16, 16, 256)          0         ['conv4_block1_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_block1_0_conv (Conv2  (None, 16, 16, 1024)         525312    ['conv3_block4_out[0][0]']    
 D)       

                                                                                                  
 conv4_block3_out (Activati  (None, 16, 16, 1024)         0         ['conv4_block3_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv4_block4_1_conv (Conv2  (None, 16, 16, 256)          262400    ['conv4_block3_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv4_block4_1_bn (BatchNo  (None, 16, 16, 256)          1024      ['conv4_block4_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_blo

 D)                                                                                               
                                                                                                  
 conv4_block6_3_bn (BatchNo  (None, 16, 16, 1024)         4096      ['conv4_block6_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block6_add (Add)      (None, 16, 16, 1024)         0         ['conv4_block5_out[0][0]',    
                                                                     'conv4_block6_3_bn[0][0]']   
                                                                                                  
 conv4_block6_out (Activati  (None, 16, 16, 1024)         0         ['conv4_block6_add[0][0]']    
 on)                                                                                              
          

 conv5_block3_1_relu (Activ  (None, 8, 8, 512)            0         ['conv5_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block3_2_conv (Conv2  (None, 8, 8, 512)            2359808   ['conv5_block3_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv5_block3_2_bn (BatchNo  (None, 8, 8, 512)            2048      ['conv5_block3_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv5_block3_2_relu (Activ  (None, 8, 8, 512)            0         ['conv5_block3_2_bn[0][0]']   
 ation)   

## Data Processing

In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 20, width_shift_range =0.1, height_shift_range = 0.1,fill_mode='nearest',
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
train_set = train_datagen.flow_from_directory('dataset/dataset/train',
                                              target_size = (256,256),
                                              batch_size = 32,
                                              class_mode = 'binary')
# print(test_datagen)

test_set = test_datagen.flow_from_directory('dataset/dataset/test',
                                            target_size = (256,256),
                                            batch_size = 32, 
                                            class_mode = 'binary')

Found 1105 images belonging to 2 classes.
Found 212 images belonging to 2 classes.


In [9]:
test_set.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [10]:
len(train_set.classes)

1105

## Modelling and Evaluation

In [11]:
batch_size = 32
EPOCH = 25

In [12]:
class timecallback(tf.keras.callbacks.Callback):
    def __init__(self):
        self.times = []
        # use this value as reference to calculate cummulative time taken
        self.timetaken = time.process_time()
    def on_epoch_end(self,epoch,logs = {}):
        self.times.append((epoch,time.process_time() - self.timetaken))
    def on_train_end(self,logs = {}):
        plt.xlabel('Epoch')
        plt.ylabel('Total time taken until an epoch in seconds')
        plt.plot(*zip(*self.times))
        plt.savefig(str(self.timetaken)+'_time.png', dpi=300, bbox_inches='tight', transparent=True)
        plt.show()

In [13]:
def modelling(mdl, name):
    batch_size = 32
    mdl_history = mdl.fit_generator(train_set,
                      epochs =EPOCH,
                      callbacks=[timecallback()],
                      steps_per_epoch = 455/batch_size,
                      validation_data = test_set)
    mdl.save(name + '.h5')
    mdl_pred(mdl)
    return mdl_history

In [14]:
def mdl_pred(mdl):
    preds = mdl.predict(test_set)
    
    scores = mdl.evaluate(test_set)
    print("%s%s: %.2f%%" % ("evaluate ",mdl.metrics_names[1], scores[1]*100))
    
    #score = mdl.evaluate_generator(test_set,preds)
    #print(" Total: ", len(test_set.filenames))
    #print("Loss: ", score[0], "Accuracy: ", score[1])

In [15]:
def evaluation_plot(vanilla_history, alex_history, google_history, resnet_history):
    plt.subplots_adjust(left=0.01,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
    plt.figure(figsize=(13,6))
    plt.tight_layout()
    handle = ['Vanilla CNN', 'AlexNet', 'GoogleNet', 'ResNet']
    dash = [(5, 1), (5, 5), (5, 10), (5, 15)]
    count = 0
    for history in [vanilla_history, alex_history, google_history, resnet_history]:
        
        # Accessing and visualizing training history
        print(history.history.keys())
        plt.suptitle("Loss")
        # Plot training and validation loss
     
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label = handle[count], linestyle='--', dashes=dash[count])
        plt.title('Training Set')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(loc='upper left')
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['val_loss'], label = handle[count], linestyle='--', dashes=dash[count])
        plt.title('Validation Set')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(loc='upper left')
        
        count = count+1
    plt.savefig('loss.png', dpi=300, bbox_inches='tight', transparent=True)
    plt.show()
    

    plt.figure(figsize=(13,6))
    plt.tight_layout()
    count = 0
    for history in [vanilla_history, alex_history, google_history, resnet_history]:
        # Plot training and validation accuracy
        
        print(history.history.keys())
        plt.suptitle("Accuracy")
        # Plot training and validation loss
     
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label = handle[count], linestyle='--', dashes=dash[count])
        plt.title('Training Set')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(loc='upper left')
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['val_accuracy'], label = handle[count], linestyle='--', dashes=dash[count])
        plt.title('Validation Set')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(loc='upper left')
        
        count = count+1
    plt.savefig('accuracy.png', dpi=300, bbox_inches='tight', transparent=True)
    plt.show()

In [16]:
# evaluation_plot(alex_history)

### Vanilla CNN

In [31]:
K.clear_session()
vanilla_history = modelling(vanilla_model, 'vanilla_CNN_model')

C:\Users\olawumi.abayomi\AppData\Local\Temp\ipykernel_19512\471258429.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  mdl_history = mdl.fit_generator(train_set,


Epoch 1/25
 1/14 [=>............................] - ETA: 2:55 - loss: 0.2465 - accuracy: 0.9062

KeyboardInterrupt: 

### AlexNet

In [ ]:
K.clear_session()
alex_history = modelling(alexnet_model, 'alexnet_model')

### GoogleNet

In [ ]:
K.clear_session()
googlenet_history = modelling(googlenet_model, 'googlenet_model')

### ResNet

In [ ]:
K.clear_session()
resnet_history = modelling(resnet_model, 'resnet_model')

In [ ]:
evaluation_plot(vanilla_history, alex_history, googlenet_history, resnet_history)

In [ ]:
class_list = ['Not-Glaucoma','Glaucoma']

In [ ]:
test_set.classes

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(20,20))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    cb = plt.colorbar()
    cb.remove() 
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def plot_confusion_mtx(model, name):
    #Confution Matrix
    Y_pred = model.predict_generator(test_set) > 0.5
    y_pred = Y_pred
    # np.argmax(Y_pred, axis=1)
    # [f[0] for f in Y_pred]
    print(Y_pred)
    print(y_pred)
    print('Confusion Matrix')
    cm = confusion_matrix(test_set.classes, y_pred)
    #disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_list)
    #disp.plot()
    
    sns.heatmap(cm, square=True, annot=True, fmt='d', cbar=False, cmap='Blues',
            xticklabels=class_list, yticklabels=class_list)
 
    plt.xlabel('Predicted label')
    plt.ylabel('Actual label')
    #plot_confusion_matrix(cm, class_list, title='Confusion Matrix', classes = class_list)
    #Print Classification Report
    print('Classification Report')
    print(classification_report(test_set.classes, y_pred))
    plt.savefig(name +'.png', dpi=300)

#### Vanilla CNN Confusion Matrix

#### AlexNet Confusion Matrix

In [ ]:
plot_confusion_mtx(vanilla_model, 'vanilla_cnn_cfx')

In [ ]:
plot_confusion_mtx(alexnet_model, 'alexnet_cfx')

#### GoogleNet Confusion Matrix

In [ ]:
plot_confusion_mtx(googlenet_model, 'ggnet_cfx')

#### ResNet Confusion Matrix

In [ ]:
plot_confusion_mtx(resnet_model, 'resnet_cfx')

In [ ]:

model=load_model('alexnet_model.h5')

In [ ]:
test_image = image.load_img('/content/drive/MyDrive/GlauPro/dataset/dataset/test/class1/Im256.jpg', target_size = (256,256))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
train_set.class_indices
if result[0][0] == 1:
     print("Glaucoma")
else:
     print("Not Glaucoma")


In [ ]:
test_image = image.load_img('/content/drive/MyDrive/GlauPro/dataset/dataset/test/class0/Im001.jpg', target_size = (256,256))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
train_set.class_indices
if result[0][0] == 1:
     print("Glaucoma")
else:
     print("Not Glaucoma")
